In [1]:
#https://en.wikipedia.org/wiki/Discrete_Fourier_transform_over_a_ring
#implementation of the discrete Fourier transform over a ring R
#can assume R is an integral domain
#so just choose \alpha a primitive n-th root of unity, i.e. \alpha^n=1, \alpha \ne 1
#ensure n invertible, i.e. p=char(R) does not divide n
#note x^{p-1} = 1, so we require n|p-1

In [2]:
#find an n-th root in finite field K
def primitive_root(K,n):
    for a in K:
        if a**n == 1 and all(a**k != 1 for k in range(1,n)):
            return a

In [3]:
#find an n^th root over unity in an extension F_q of F_p
#note that if p|n, n==0 mod p, so there is no n^th root over F_p^r
def primitive_root_unity(p,n):
    assert not p.divides(n)
    #find an n^th root of unity over F_q
    r = 1
    while not n.divides(p**r - 1):
        r += 1
    q = p**r
    k.<a> = GF(q)
    return a^((q-1)/n), k

In [4]:
#DFT over a ring with primitive root alpha
def fourier_transform(v,alpha):
    return [sum(v[j]*alpha**(j*k) for j in range(len(v))) for k in range(len(v))]

In [5]:
#define the inverse Fourier transform
def inverse_fourier_transform(f,alpha):
    return [K(1/len(f))*sum(f[k]*alpha**(-j*k) for k in range(len(f))) for j in range(len(f))]

In [6]:
#we are looking at the group algebra F_p[C_N]
#this is F_p[x]/(x^N-1)
#if p|N, we can write (x^N-1) = (x^m-1)^{p^s} for some s
#we are looking at roots of unity mod p
#to factor x^m-1, use cyclotomic polynomials: x^m-1 = \prod_{d|m} phi_d(x)
#to factor phi_d(x) in F_p, that's equivalent to looking at (p) in Z[x]/Phi_d(x) = Z[zeta]
#(p) = P_1...P_g where each P_i has the same residue degree f
#Here f*g = phi(d), and f is the order of p modulo d, phi is Euler totient
#phi_d(x) factors in F_p[x] into g polynomials, each of degree f
#thus F_p[x]/(x^N-1) \cong \prod_{d|m} \prod_{i=1}^g F_p[x]/(P_i^{p^s})
#by the Chinese remainder theorem
#i.e. just factor x^N-1 mod p, and map onto residue classes
#we can also factor over a splitting field, F_q
#this will result in linear factors, with possible multiplicity if p|N

In [47]:
#h is an element of F_p[C_N]
#that is, h = h0+h1*x+h2x^2+...+h_{N-1}x^N-1
#we allow h as a list of N numbers modulo p
#h = [h0,h1,h2,...,h_{N-1}]
def discrete_fourier_transform(h,p,splitting_field=False):
    #length of list is size of N
    N = len(h)
    #define the polynomial ring F_p[x]
    R = PolynomialRing(GF(p),'x')
    #name the generator x an element of R
    x = R.0
    #define the polynomial x^N-1
    f = x**N-1; assert f in R
    if splitting_field:
        K.<a> = f.splitting_field()
        #define the polynomial ring over extended base field
        R = PolynomialRing(K,'x')
        #name the generator x an element of R
        x = R.0
        #define the polynomial x^N-1
        f = x**N-1; assert f in R
    #define the quotient ring F_p[x]/(x^N-1)
    S = R.quotient(x^N - 1, 'x')
    #transform the list of coefficients of h into a polynomial in R=F_p[x]
    h = sum(h[i]*x**i for i in range(N)); assert h in S
    #factor f in F_p[x], save as list of factors and multiplicities
    f_factors = list(f.factor())
    #implement the Chinese remainder theorem mapping S=F_p[x]/(x^N-1) --> \prod_i R/(factor_i^mult_i)
    h_transform=[list(R.quotient(f_factors[i][0]**f_factors[i][1])(h)) for i in range(len(f_factors))]
    return h_transform

In [8]:
def dft_matrix(p,n):
    return matrix([flatten(discrete_fourier_transform([1 if i==j else 0 for j in range(n)],p,splitting_field=True)) for i in range(n)]).transpose()

In [9]:
#compute the DFT matrix as a Vandermonde matrix over F_q
#find a primitive n^th root of unity over F_q as long as n == 0 over F_p
def dft_matrix_vandermonde(p,n):
    alpha, k = primitive_root_unity(p,n)
    return matrix(k, [[alpha^(i*j) for i in range(n)] for j in range(n)])

In [63]:
dft_matrix_vandermonde(3,4)

[      1       1       1       1]
[      1   a + 1       2 2*a + 2]
[      1       2       1       2]
[      1 2*a + 2       2   a + 1]

In [54]:
dft_matrix(3,4).base_ring().order()

9

In [58]:
dft_matrix(3,4).base_ring().gen()

a

In [61]:
[print(k) for k in range(100) if dft_matrix(3,5)^k == identity_matrix(5)]

0
80


[None, None]

In [112]:
dft_matrix_vandermonde(3,11).base_ring()

Finite Field in a of size 3^5

In [94]:
dft_matrix_vandermonde(3,25).charpoly().factor()

(x + 2)^6 * (x + a^19 + 2*a^18 + 2*a^17 + a^15 + 2*a^14 + 2*a^13 + 2*a^12 + a^10 + a^9 + 2*a^8 + a^7 + 2*a^4 + 2*a^3 + a^2 + 1)^6 * (x + 2*a^19 + a^18 + a^17 + 2*a^15 + a^14 + a^13 + a^12 + 2*a^10 + 2*a^9 + a^8 + 2*a^7 + a^4 + a^3 + 2*a^2 + 2)^6 * (x + 1)^7

In [125]:
#it appears the eigenvalues are 1, -1, and two elements of order 4, just as in the complex case. 
k.<a> = dft_matrix_vandermonde(3,11).charpoly().splitting_field()
print(k)
eigs = matrix(k,dft_matrix_vandermonde(3,11)).eigenvalues(extend=False); print(eigs)
print(len(set(eigs)))
[eig.multiplicative_order() for eig in eigs]

Finite Field in a of size 3^10
[1, 1, 2, 2, 2, 2*a^9 + 2*a^7 + a^6 + 2*a^4 + 2*a^3 + a^2 + 2*a + 1, 2*a^9 + 2*a^7 + a^6 + 2*a^4 + 2*a^3 + a^2 + 2*a + 1, 2*a^9 + 2*a^7 + a^6 + 2*a^4 + 2*a^3 + a^2 + 2*a + 1, a^9 + a^7 + 2*a^6 + a^4 + a^3 + 2*a^2 + a + 2, a^9 + a^7 + 2*a^6 + a^4 + a^3 + 2*a^2 + a + 2, a^9 + a^7 + 2*a^6 + a^4 + a^3 + 2*a^2 + a + 2]
4


[1, 1, 2, 2, 2, 4, 4, 4, 4, 4, 4]

In [137]:
a^(-(3^10-1)/4)

a^9 + a^7 + 2*a^6 + a^4 + a^3 + 2*a^2 + a + 2

In [11]:
dft_matrix_vandermonde(3,4).base_ring().gen()^6

2*a + 2

In [12]:
dft_matrix_vandermonde(3,4).det()

2*a + 2

In [13]:
dft_matrix(3,4).det()

a + 1

In [14]:
k.<a> = dft_matrix(3,5).charpoly().splitting_field()
matrix(k,dft_matrix(3,5)).eigenvalues(extend=False)

[a, a + 1, 2*a^3 + 1, 2*a^3 + 2*a^2 + 2, a^3 + a^2 + 1]

In [15]:
dft_matrix(3,4).charpoly().factor()

(x + 2*a + 2) * (x^3 + (2*a + 1)*x^2 + 2*a*x + 2)

In [16]:
dft_matrix_vandermonde(3,5).charpoly().roots()

[(2, 1), (1, 1), (a^3 + a^2 + 1, 1), (2*a^3 + 2*a^2 + 2, 2)]

In [17]:
k.<a> = dft_matrix_vandermonde(3,5).charpoly().splitting_field()
matrix(k,dft_matrix_vandermonde(3,5)).eigenvalues(extend=False)

[2, 1, a^3 + a^2 + 1, 2*a^3 + 2*a^2 + 2, 2*a^3 + 2*a^2 + 2]

In [18]:
discrete_fourier_transform([0, 1, 0, 0],3,splitting_field=True)

[[2], [1], [2*a + 2], [a + 1]]

In [19]:
#find the \sqrt{n} in a splitting field of x^n-1 over F_p
def square_root_splitting_field(p,n):
    #define the polynomial ring F_p[x]
    R = PolynomialRing(GF(p),'x')
    #name the generator x an element of R
    x = R.0
    #define the polynomial x^N-1
    f = x**n-1; assert f in R
    #find the splitting field 
    K.<a> = f.splitting_field()
    #embed n in the field K, and take the square root
    return sqrt(K(n))

In [20]:
dft_matrix(5,6)^120

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

In [21]:
dft_matrix(3,4).charpoly().splitting_field('a')

Finite Field in a of size 3^6

In [22]:
def inv_discrete_fourier_transform(hhat,p,splitting_field=False):
    N = sum(len(l) for l in hhat)
    #define the polynomial ring F_p[x]
    R = PolynomialRing(GF(p),'x')
    #name the generator x an element of R
    x = R.0
    #define the polynomial x^N-1
    f = x**N-1; assert f in R
    if splitting_field:
        K.<a> = f.splitting_field()
        #define the polynomial ring over extended base field
        R = PolynomialRing(K,'x')
        #name the generator x an element of R
        x = R.0
        #define the polynomial x^N-1
        f = x**N-1; assert f in R
    S = R.quotient(x^N - 1, 'x')
    f_factors = list(f.factor())
    #perform inverse of Chinese remainder theorem
    #for each modulus N_i = N/n_i, where n_i is the modulus of each factor
    #Bezout's theorem applies, so we get M_i*N_i + m_i*n_i = 1
    #a solution x = \sum_{i=1}^k a_i*M_i*N_i, where a_i are the remainders
    n = [f_factors[i][0]**f_factors[i][1] for i in range(len(f_factors))]
    #get coefficients M_i, m_i from N_i, n_i
    M = [xgcd(f/n[i],n[i])[1] for i in range(len(n))]
    #get remainders as polynomials in R
    a = [sum(hhat[i][j]*x**j for j in range(len(hhat[i]))) for i in range(len(hhat))]
    inv_transform = sum(a[i]*M[i]*(f/n[i]) for i in range(len(a)))
    return list(S(inv_transform))

In [23]:
inv_discrete_fourier_transform([[4], [1], [4, 4], [1, 4]],5)

[0, 0, 0, 0, 0, 1]

In [24]:
n=6; p=7
#finite field of size p
K = GF(p)
assert K(n) != K(0) #ensure n is invertible
assert n.divides(p-1) #ensure a primitive n-th root of unity exists
#list to be transformed
v = [K(1) for i in range(n)]
alpha = primitive_root(K,n)
f=fourier_transform(v,alpha); f
inverse_fourier_transform(f,alpha)

[1, 1, 1, 1, 1, 1]

In [25]:
from sage.misc.flatten import flatten
#idempotent corresponding to (1,0) in the product of quotient rings for p=3, N=6
quotient_idem_0 = [[1,0,0],[0,0,0]]
#idempotent corresponding to (0,1) in the product of quotient rings for p=3, N=6
quotient_idem_1 = [[0,0,0],[1,0,0]]
N = len(flatten(quotient_idem_0))
R = PolynomialRing(GF(3),'x')
S = R.quotient(x^N - 1, 'x')
x = R.0 #name the generator x an element of R
inv_FT_0 = inv_discrete_fourier_transform(quotient_idem_0,3,splitting_field=False) #inverse FT
idem_0 = sum(inv_FT_0[i]*x**i for i in range(N)); assert idem_0 in S #map list to poly in S
inv_FT_1 = inv_discrete_fourier_transform(quotient_idem_1,3,splitting_field=False) #inverse DFT 
idem_1 = sum(inv_FT_1[i]*x**i for i in range(N)); assert idem_1 in S #map list to poly in S
print(idem_0); print(idem_1)

x^3 + 2
2*x^3 + 2
